In [1]:
from google.colab import files
import zipfile, os

# Upload your dataset.zip
uploaded = files.upload()

# Extract
zip_path = list(uploaded.keys())[0]
extract_dir = "dataset"
os.makedirs(extract_dir, exist_ok=True)

with zipfile.ZipFile(zip_path, 'r') as zip_ref:
    zip_ref.extractall(extract_dir)

print("✅ Files extracted to:", extract_dir)
!ls dataset


Saving dataset.zip to dataset.zip
✅ Files extracted to: dataset
dataset


In [2]:
import os

base_dir = "dataset"  # extracted folder

# Walk through the dataset and list files
for root, dirs, files in os.walk(base_dir):
    print(f"📂 {root} -> {len(files)} files")


📂 dataset -> 0 files
📂 dataset/dataset -> 0 files
📂 dataset/dataset/dataset -> 1 files
📂 dataset/dataset/dataset/akbar-allahabadi -> 0 files
📂 dataset/dataset/dataset/akbar-allahabadi/ur -> 50 files
📂 dataset/dataset/dataset/akbar-allahabadi/hi -> 50 files
📂 dataset/dataset/dataset/akbar-allahabadi/en -> 50 files
📂 dataset/dataset/dataset/habib-jalib -> 1 files
📂 dataset/dataset/dataset/habib-jalib/ur -> 50 files
📂 dataset/dataset/dataset/habib-jalib/hi -> 50 files
📂 dataset/dataset/dataset/habib-jalib/en -> 50 files
📂 dataset/dataset/dataset/nida-fazli -> 0 files
📂 dataset/dataset/dataset/nida-fazli/ur -> 50 files
📂 dataset/dataset/dataset/nida-fazli/hi -> 50 files
📂 dataset/dataset/dataset/nida-fazli/en -> 50 files
📂 dataset/dataset/dataset/bahadur-shah-zafar -> 0 files
📂 dataset/dataset/dataset/bahadur-shah-zafar/ur -> 50 files
📂 dataset/dataset/dataset/bahadur-shah-zafar/hi -> 50 files
📂 dataset/dataset/dataset/bahadur-shah-zafar/en -> 50 files
📂 dataset/dataset/dataset/allama-iqba

In [3]:
import os, re
import pandas as pd
from sklearn.model_selection import train_test_split
from tokenizers import ByteLevelBPETokenizer

# STEP 1: Urdu Cleaning

DIACRITICS = re.compile("[\u064B-\u0652]")

def clean_urdu_text(text: str) -> str:
    if not isinstance(text, str):
        return ""
    text = text.replace("ى", "ی").replace("ي", "ی").replace("ئ", "ی")
    text = text.replace("ه", "ہ").replace("ھ", "ہ")
    text = text.replace("ك", "ک")
    text = text.replace("أ", "ا").replace("إ", "ا").replace("آ", "ا")
    text = re.sub(DIACRITICS, "", text)
    text = re.sub(r"[^\u0600-\u06FF\s]", "", text)
    return text.strip()


# STEP 2: Build Urdu–Roman Pairs

base_dir = "dataset/dataset/dataset"  # extracted root
pairs = []

for poet in os.listdir(base_dir):
    poet_path = os.path.join(base_dir, poet)
    ur_path = os.path.join(poet_path, "ur")
    en_path = os.path.join(poet_path, "en")

    if os.path.exists(ur_path) and os.path.exists(en_path):
        ur_files = sorted(os.listdir(ur_path))
        en_files = sorted(os.listdir(en_path))

        for ur_file, en_file in zip(ur_files, en_files):
            try:
                with open(os.path.join(ur_path, ur_file), "r", encoding="utf-8") as f1:
                    ur_text = f1.read().strip()
                with open(os.path.join(en_path, en_file), "r", encoding="utf-8") as f2:
                    en_text = f2.read().strip()

                ur_text = clean_urdu_text(ur_text)
                if ur_text and en_text:
                    pairs.append((ur_text, en_text))
            except Exception as e:
                print("⚠️ Error in", ur_file, e)

df = pd.DataFrame(pairs, columns=["Urdu", "Roman"])
print("✅ Dataset built with shape:", df.shape)
print(df.head())


✅ Dataset built with shape: (1314, 2)
                                                Urdu  \
0  اہ جو دل سے نکالی جایے گی \nکیا سمجہتے ہو کہ خ...   
1  اج ارایش گیسویے دوتا ہوتی ہے \nپہر مری جان گرف...   
2  انکہیں مجہے تلووں سے وہ ملنے نہیں دیتے \nارمان...   
3  اگر دل واقف نیرنگیٔ طبع صنم ہوتا \nزمانہ کی دو...   
4  اپنے پہلو سے وہ غیروں کو اٹہا ہی نہ سکے \nان ک...   

                                               Roman  
0  aah jo dil se nikālī jā.egī \nkyā samajhte ho ...  
1  aaj ārā.ish-e-gesū-e-dotā hotī hai \nphir mirī...  
2  āñkheñ mujhe talvoñ se vo malne nahīñ dete \na...  
3  agar dil vāqif-e-nairañgī-e-tab-e-sanam hotā \...  
4  apne pahlū se vo ġhairoñ ko uThā hī na sake \n...  


In [4]:

# STEP 3: Train Tokenizers

os.makedirs("tokenizers", exist_ok=True)

df["Urdu"].to_csv("urdu_corpus.txt", index=False, header=False)
df["Roman"].to_csv("roman_corpus.txt", index=False, header=False)

# Train Urdu tokenizer
urdu_tok = ByteLevelBPETokenizer()
urdu_tok.train(files="urdu_corpus.txt", vocab_size=5000, min_frequency=2)
urdu_tok.save_model("tokenizers", "urdu")

# Train Roman tokenizer
roman_tok = ByteLevelBPETokenizer()
roman_tok.train(files="roman_corpus.txt", vocab_size=5000, min_frequency=2)
roman_tok.save_model("tokenizers", "roman")

print("✅ Tokenizers trained and saved in /tokenizers")


✅ Tokenizers trained and saved in /tokenizers


In [5]:

# STEP 4: Train/Val/Test Split

os.makedirs("processed_data", exist_ok=True)

train, temp = train_test_split(df, test_size=0.5, random_state=42)
val, test = train_test_split(temp, test_size=0.5, random_state=42)

train.to_csv("processed_data/train.csv", index=False)
val.to_csv("processed_data/val.csv", index=False)
test.to_csv("processed_data/test.csv", index=False)

print("✅ Data saved in /processed_data:")
print("Train:", train.shape, " Val:", val.shape, " Test:", test.shape)


✅ Data saved in /processed_data:
Train: (657, 2)  Val: (328, 2)  Test: (329, 2)


In [6]:
import torch
import torch.nn as nn


# Encoder: 2-layer BiLSTM

class Encoder(nn.Module):
    def __init__(self, input_dim, emb_dim=256, hid_dim=512, n_layers=2, dropout=0.3):
        super().__init__()
        self.embedding = nn.Embedding(input_dim, emb_dim)
        self.bilstm = nn.LSTM(
            input_size=emb_dim,
            hidden_size=hid_dim,
            num_layers=n_layers,
            dropout=dropout,
            bidirectional=True,
            batch_first=True
        )
        # Combine forward and backward hidden states
        self.fc_hidden = nn.Linear(hid_dim * 2, hid_dim)
        self.fc_cell = nn.Linear(hid_dim * 2, hid_dim)
        self.dropout = nn.Dropout(dropout)

    def forward(self, src):
        # src: [batch_size, seq_len]
        embedded = self.dropout(self.embedding(src))  # [batch, seq_len, emb_dim]
        outputs, (hidden, cell) = self.bilstm(embedded)

        # Take last forward and backward hidden states
        hidden = torch.tanh(self.fc_hidden(torch.cat((hidden[-2], hidden[-1]), dim=1)))
        cell = torch.tanh(self.fc_cell(torch.cat((cell[-2], cell[-1]), dim=1)))

        return outputs, hidden, cell



# Decoder: 4-layer LSTM

class Decoder(nn.Module):
    def __init__(self, output_dim, emb_dim=256, hid_dim=512, n_layers=4, dropout=0.3):
        super().__init__()
        self.output_dim = output_dim
        self.embedding = nn.Embedding(output_dim, emb_dim)
        self.lstm = nn.LSTM(
            input_size=emb_dim,
            hidden_size=hid_dim,
            num_layers=n_layers,
            dropout=dropout,
            batch_first=True
        )
        self.fc_out = nn.Linear(hid_dim, output_dim)
        self.dropout = nn.Dropout(dropout)

    def forward(self, input, hidden, cell):
        # input: [batch_size] -> reshape for LSTM
        input = input.unsqueeze(1)  # [batch, 1]
        embedded = self.dropout(self.embedding(input))  # [batch, 1, emb_dim]

        output, (hidden, cell) = self.lstm(embedded, (hidden, cell))
        prediction = self.fc_out(output.squeeze(1))  # [batch, output_dim]

        return prediction, hidden, cell



# Seq2Seq Wrapper

class Seq2Seq(nn.Module):
    def __init__(self, encoder, decoder, device, dec_layers=4):
        super().__init__()
        self.encoder = encoder
        self.decoder = decoder
        self.device = device
        self.dec_layers = dec_layers

    def forward(self, src, trg, teacher_forcing_ratio=0.5):
        batch_size = src.shape[0]
        trg_len = trg.shape[1]
        trg_vocab_size = self.decoder.output_dim

        outputs = torch.zeros(batch_size, trg_len, trg_vocab_size).to(self.device)

        # Encoder forward
        _, hidden, cell = self.encoder(src)  # hidden/cell: [batch, hid_dim]

        # Expand hidden & cell to match decoder's number of layers
        hidden = hidden.unsqueeze(0).repeat(self.dec_layers, 1, 1)
        cell = cell.unsqueeze(0).repeat(self.dec_layers, 1, 1)

        # First input to decoder is <sos>
        input = trg[:, 0]

        for t in range(1, trg_len):
            output, hidden, cell = self.decoder(input, hidden, cell)
            outputs[:, t, :] = output

            # Determine next input (teacher forcing)
            top1 = output.argmax(1)
            input = trg[:, t] if torch.rand(1).item() < teacher_forcing_ratio else top1

        return outputs


In [7]:
from tokenizers import ByteLevelBPETokenizer

# Load tokenizers
urdu_tokenizer = ByteLevelBPETokenizer("tokenizers/urdu-vocab.json",
                                       "tokenizers/urdu-merges.txt")
roman_tokenizer = ByteLevelBPETokenizer("tokenizers/roman-vocab.json",
                                        "tokenizers/roman-merges.txt")

# Define input/output dimensions for model
INPUT_DIM = urdu_tokenizer.get_vocab_size()
OUTPUT_DIM = roman_tokenizer.get_vocab_size()

print("✅ Urdu vocab size:", INPUT_DIM)
print("✅ Roman vocab size:", OUTPUT_DIM)


✅ Urdu vocab size: 5000
✅ Roman vocab size: 5000


In [8]:
EMB_DIM = 256
HID_DIM = 512
ENC_LAYERS = 2
DEC_LAYERS = 4
DROPOUT = 0.3
DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Initialize encoder and decoder
encoder = Encoder(INPUT_DIM, EMB_DIM, HID_DIM, n_layers=ENC_LAYERS, dropout=DROPOUT)
decoder = Decoder(OUTPUT_DIM, EMB_DIM, HID_DIM, n_layers=DEC_LAYERS, dropout=DROPOUT)

# Initialize Seq2Seq model
model = Seq2Seq(encoder, decoder, device=DEVICE, dec_layers=DEC_LAYERS).to(DEVICE)

print("✅ Seq2Seq model initialized:")
print(model)


✅ Seq2Seq model initialized:
Seq2Seq(
  (encoder): Encoder(
    (embedding): Embedding(5000, 256)
    (bilstm): LSTM(256, 512, num_layers=2, batch_first=True, dropout=0.3, bidirectional=True)
    (fc_hidden): Linear(in_features=1024, out_features=512, bias=True)
    (fc_cell): Linear(in_features=1024, out_features=512, bias=True)
    (dropout): Dropout(p=0.3, inplace=False)
  )
  (decoder): Decoder(
    (embedding): Embedding(5000, 256)
    (lstm): LSTM(256, 512, num_layers=4, batch_first=True, dropout=0.3)
    (fc_out): Linear(in_features=512, out_features=5000, bias=True)
    (dropout): Dropout(p=0.3, inplace=False)
  )
)


In [9]:
import torch
from torch.utils.data import Dataset, DataLoader
from tokenizers import ByteLevelBPETokenizer
import pandas as pd

# Load Tokenizers

urdu_tokenizer = ByteLevelBPETokenizer("tokenizers/urdu-vocab.json",
                                       "tokenizers/urdu-merges.txt")
roman_tokenizer = ByteLevelBPETokenizer("tokenizers/roman-vocab.json",
                                        "tokenizers/roman-merges.txt")

# Special tokens
SOS_IDX = 0   # <s>
EOS_IDX = 1   # </s>
PAD_IDX = 2   # <pad>


# Decode function

def decode_tokens(token_ids, tokenizer):
    """Convert token IDs to text using ByteLevelBPETokenizer."""
    if isinstance(token_ids, torch.Tensor):
        token_ids = token_ids.tolist()
    # Remove special tokens
    token_ids = [idx for idx in token_ids if idx not in [PAD_IDX, SOS_IDX, EOS_IDX]]
    return tokenizer.decode(token_ids, skip_special_tokens=True)


# Custom Dataset Class

class TranslationDataset(Dataset):
    def __init__(self, csv_file, urdu_tokenizer, roman_tokenizer, max_len=100):
        self.data = pd.read_csv(csv_file)
        self.urdu_tokenizer = urdu_tokenizer
        self.roman_tokenizer = roman_tokenizer
        self.max_len = max_len

    def __len__(self):
        return len(self.data)

    def encode_text(self, text, tokenizer, add_special_tokens=True):
        tokens = tokenizer.encode(text).ids
        if add_special_tokens:
            tokens = [SOS_IDX] + tokens + [EOS_IDX]
        return tokens

    def pad_sequence(self, seq):
        if len(seq) < self.max_len:
            seq = seq + [PAD_IDX] * (self.max_len - len(seq))
        else:
            seq = seq[:self.max_len]
        return seq

    def __getitem__(self, idx):
        urdu = self.data.iloc[idx]["Urdu"]
        roman = self.data.iloc[idx]["Roman"]

        # Encode & pad
        src_seq = self.encode_text(urdu, self.urdu_tokenizer)
        trg_seq = self.encode_text(roman, self.roman_tokenizer)

        src_seq = self.pad_sequence(src_seq)
        trg_seq = self.pad_sequence(trg_seq)

        return torch.tensor(src_seq), torch.tensor(trg_seq)

# Build DataLoaders

def get_dataloaders(batch_size=32, max_len=100):
    train_dataset = TranslationDataset("processed_data/train.csv", urdu_tokenizer, roman_tokenizer, max_len)
    val_dataset   = TranslationDataset("processed_data/val.csv", urdu_tokenizer, roman_tokenizer, max_len)
    test_dataset  = TranslationDataset("processed_data/test.csv", urdu_tokenizer, roman_tokenizer, max_len)

    train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
    val_loader   = DataLoader(val_dataset, batch_size=batch_size)
    test_loader  = DataLoader(test_dataset, batch_size=batch_size)

    return train_loader, val_loader, test_loader


In [10]:
train_loader, val_loader, test_loader = get_dataloaders(batch_size=32, max_len=100)

# Check one batch
for src, trg in train_loader:
    print("Source batch shape:", src.shape)  # [batch, seq_len]
    print("Target batch shape:", trg.shape)  # [batch, seq_len]
    break


Source batch shape: torch.Size([32, 100])
Target batch shape: torch.Size([32, 100])


In [14]:
import torch
import torch.nn as nn
import torch.optim as optim


# Loss Function & Optimizer

PAD_IDX = 2  # from dataset.py

criterion = nn.CrossEntropyLoss(ignore_index=PAD_IDX)  # ignore padding in loss
optimizer = optim.Adam(model.parameters(), lr=1e-3)



# Training Function

def train(model, dataloader, optimizer, criterion, clip=1):
    model.train()
    epoch_loss = 0

    for src, trg in dataloader:
        src, trg = src.to(DEVICE), trg.to(DEVICE)

        optimizer.zero_grad()

        # Forward pass
        output = model(src, trg)

        # output: [batch, trg_len, output_dim]
        # reshape for loss: [batch*trg_len, output_dim]
        output_dim = output.shape[-1]
        output = output[:,1:,:].reshape(-1, output_dim)
        trg = trg[:,1:].reshape(-1)

        loss = criterion(output, trg)
        loss.backward()

        # gradient clipping
        torch.nn.utils.clip_grad_norm_(model.parameters(), clip)
        optimizer.step()

        epoch_loss += loss.item()

    return epoch_loss / len(dataloader)


# Evaluation Function

def evaluate(model, dataloader, criterion):
    model.eval()
    epoch_loss = 0

    with torch.no_grad():
        for src, trg in dataloader:
            src, trg = src.to(DEVICE), trg.to(DEVICE)

            output = model(src, trg, teacher_forcing_ratio=0)  # no teacher forcing in eval

            output_dim = output.shape[-1]
            output = output[:,1:,:].reshape(-1, output_dim)
            trg = trg[:,1:].reshape(-1)

            loss = criterion(output, trg)
            epoch_loss += loss.item()

    return epoch_loss / len(dataloader)


In [16]:
import os
import torch

# Checkpoints folder
checkpoint_dir = "/content/drive/MyDrive/urdu_roman_checkpoints"
os.makedirs(checkpoint_dir, exist_ok=True)

N_EPOCHS = 10
CLIP = 1
best_valid_loss = float('inf')

train_loader, val_loader, test_loader = get_dataloaders(batch_size=32, max_len=100)

for epoch in range(N_EPOCHS):
    train_loss = train(model, train_loader, optimizer, criterion, CLIP)
    valid_loss = evaluate(model, val_loader, criterion)

    # Save checkpoint
    checkpoint_path = os.path.join(checkpoint_dir, f"epoch_{epoch+1}.pt")
    torch.save(model.state_dict(), checkpoint_path)

    # Save best model
    if valid_loss < best_valid_loss:
        best_valid_loss = valid_loss
        best_model_path = os.path.join(checkpoint_dir, "best-model.pt")
        torch.save(model.state_dict(), best_model_path)

    print(f"Epoch {epoch+1}/{N_EPOCHS}")
    print(f"   Train Loss: {train_loss:.3f}")
    print(f"   Valid Loss: {valid_loss:.3f}")
    print(f"   Checkpoint saved: {checkpoint_path}")


Epoch 1/10
   Train Loss: 6.780
   Valid Loss: 6.288
   Checkpoint saved: /content/drive/MyDrive/urdu_roman_checkpoints/epoch_1.pt
Epoch 2/10
   Train Loss: 6.174
   Valid Loss: 6.244
   Checkpoint saved: /content/drive/MyDrive/urdu_roman_checkpoints/epoch_2.pt
Epoch 3/10
   Train Loss: 6.133
   Valid Loss: 6.228
   Checkpoint saved: /content/drive/MyDrive/urdu_roman_checkpoints/epoch_3.pt
Epoch 4/10
   Train Loss: 6.117
   Valid Loss: 6.228
   Checkpoint saved: /content/drive/MyDrive/urdu_roman_checkpoints/epoch_4.pt
Epoch 5/10
   Train Loss: 6.105
   Valid Loss: 6.228
   Checkpoint saved: /content/drive/MyDrive/urdu_roman_checkpoints/epoch_5.pt
Epoch 6/10
   Train Loss: 6.098
   Valid Loss: 6.229
   Checkpoint saved: /content/drive/MyDrive/urdu_roman_checkpoints/epoch_6.pt
Epoch 7/10
   Train Loss: 6.091
   Valid Loss: 6.230
   Checkpoint saved: /content/drive/MyDrive/urdu_roman_checkpoints/epoch_7.pt
Epoch 8/10
   Train Loss: 6.082
   Valid Loss: 6.228
   Checkpoint saved: /content/

In [17]:
from nltk.translate.bleu_score import corpus_bleu

def compute_bleu(model, dataloader, urdu_tokenizer, roman_tokenizer, max_len=100):
    model.eval()
    references = []
    hypotheses = []

    with torch.no_grad():
        for src, trg in dataloader:
            # Move tensors to correct device
            src, trg = src.to(DEVICE), trg.to(DEVICE)

            outputs = model(src, trg, teacher_forcing_ratio=0)
            preds = outputs.argmax(-1).cpu().numpy()
            trgs = trg.cpu().numpy()

            for i in range(len(preds)):
                ref = decode_tokens(trgs[i], roman_tokenizer)
                hyp = decode_tokens(preds[i], roman_tokenizer)

                if ref.strip():
                    references.append([ref.split()])
                    hypotheses.append(hyp.split())

    bleu = corpus_bleu(references, hypotheses)
    return bleu


In [21]:
!pip install editdistance
import editdistance


In [22]:
def compute_cer(model, dataloader, roman_tokenizer):
    model.eval()
    total_ed = 0
    total_len = 0

    with torch.no_grad():
        for src, trg in dataloader:
            src, trg = src.to(DEVICE), trg.to(DEVICE)

            outputs = model(src, trg, teacher_forcing_ratio=0)
            preds = outputs.argmax(-1).cpu().numpy()
            trgs = trg.cpu().numpy()

            for i in range(len(preds)):
                ref = decode_tokens(trgs[i], roman_tokenizer)
                hyp = decode_tokens(preds[i], roman_tokenizer)

                total_ed += editdistance.eval(hyp, ref)
                total_len += len(ref)

    cer = total_ed / total_len if total_len > 0 else 0
    return cer


In [23]:
import math

def calculate_perplexity(loss):
    """Convert average cross-entropy loss to perplexity."""
    return math.exp(loss)


In [24]:
# ================================
# Cell 17: Load Best Model & Evaluate
# ================================
# Make sure DEVICE is set
DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Load best model checkpoint
best_model_path = "/content/drive/MyDrive/urdu_roman_checkpoints/best-model.pt"
model.load_state_dict(torch.load(best_model_path, map_location=DEVICE))
model.to(DEVICE)
model.eval()

# Evaluate on test set
test_loss = evaluate(model, test_loader, criterion)
perplexity = calculate_perplexity(test_loss)

bleu = compute_bleu(model, test_loader, urdu_tokenizer, roman_tokenizer)
cer = compute_cer(model, test_loader, roman_tokenizer)

# Print results
print("\n📊 Final Evaluation Results")
print(f"✅ Test Loss     : {test_loss:.3f}")
print(f"✅ Perplexity    : {perplexity:.3f}")
print(f"✅ BLEU Score    : {bleu:.3f}")
print(f"✅ CER           : {cer:.3f}")



📊 Final Evaluation Results
✅ Test Loss     : 6.207
✅ Perplexity    : 496.292
✅ BLEU Score    : 0.000
✅ CER           : 0.823


In [25]:
import torch
import torch.nn as nn
import torch.optim as optim
import pandas as pd

CLIP = 1

experiments = [
    {
        "name": "exp1_baseline",
        "emb_dim": 256, "hid_dim": 512,
        "enc_layers": 2, "dec_layers": 4,
        "dropout": 0.3, "lr": 1e-3,
        "batch_size": 32
    },
    {
        "name": "exp2_small_fast",
        "emb_dim": 128, "hid_dim": 256,
        "enc_layers": 2, "dec_layers": 3,
        "dropout": 0.3, "lr": 5e-4,
        "batch_size": 64
    },
    {
        "name": "exp3_big_reg",
        "emb_dim": 512, "hid_dim": 512,
        "enc_layers": 3, "dec_layers": 4,
        "dropout": 0.5, "lr": 1e-4,
        "batch_size": 32
    }
]

results = []

for spec in experiments:
    print(f"\n🚀 Running {spec['name']} ...")

    # Load data
    train_loader, val_loader, test_loader = get_dataloaders(batch_size=spec["batch_size"], max_len=100)

    # Build model
    encoder = Encoder(INPUT_DIM, spec["emb_dim"], spec["hid_dim"],
                      n_layers=spec["enc_layers"], dropout=spec["dropout"])
    decoder = Decoder(OUTPUT_DIM, spec["emb_dim"], spec["hid_dim"],
                      n_layers=spec["dec_layers"], dropout=spec["dropout"])
    model = Seq2Seq(encoder, decoder, DEVICE, dec_layers=spec["dec_layers"]).to(DEVICE)

    criterion = nn.CrossEntropyLoss(ignore_index=PAD_IDX)
    optimizer = optim.Adam(model.parameters(), lr=spec["lr"])

    best_valid_loss = float('inf')

    for epoch in range(10):
        train_loss = train(model, train_loader, optimizer, criterion, CLIP)
        valid_loss = evaluate(model, val_loader, criterion)

        if valid_loss < best_valid_loss:
            best_valid_loss = valid_loss
            torch.save(model.state_dict(), f"{spec['name']}.pt")

        print(f"Epoch {epoch+1}/10 | Train: {train_loss:.3f} | Val: {valid_loss:.3f}")

    # Load best model
    model.load_state_dict(torch.load(f"{spec['name']}.pt"))
    model.eval()

    # Final evaluation
    test_loss = evaluate(model, test_loader, criterion)
    ppl = calculate_perplexity(test_loss)
    bleu = compute_bleu(model, test_loader, urdu_tokenizer, roman_tokenizer)
    cer = compute_cer(model, test_loader, roman_tokenizer)

    results.append({
        "Experiment": spec["name"],
        "Test Loss": round(test_loss, 3),
        "Perplexity": round(ppl, 3),
        "BLEU": round(bleu, 3),
        "CER": round(cer, 3)
    })

print("\n📊 Experiment Results")
df_results = pd.DataFrame(results)
print(df_results)



🚀 Running exp1_baseline ...
Epoch 1/10 | Train: 6.773 | Val: 6.298
Epoch 2/10 | Train: 6.176 | Val: 6.244
Epoch 3/10 | Train: 6.134 | Val: 6.230
Epoch 4/10 | Train: 6.116 | Val: 6.227
Epoch 5/10 | Train: 6.105 | Val: 6.223
Epoch 6/10 | Train: 6.094 | Val: 6.222
Epoch 7/10 | Train: 6.086 | Val: 6.226
Epoch 8/10 | Train: 6.081 | Val: 6.231
Epoch 9/10 | Train: 6.077 | Val: 6.238
Epoch 10/10 | Train: 6.073 | Val: 6.233

🚀 Running exp2_small_fast ...
Epoch 1/10 | Train: 8.355 | Val: 7.562
Epoch 2/10 | Train: 6.914 | Val: 6.378
Epoch 3/10 | Train: 6.186 | Val: 6.220
Epoch 4/10 | Train: 6.105 | Val: 6.215
Epoch 5/10 | Train: 6.087 | Val: 6.207
Epoch 6/10 | Train: 6.082 | Val: 6.206
Epoch 7/10 | Train: 6.065 | Val: 6.204
Epoch 8/10 | Train: 6.069 | Val: 6.205
Epoch 9/10 | Train: 6.074 | Val: 6.206
Epoch 10/10 | Train: 6.059 | Val: 6.205

🚀 Running exp3_big_reg ...
Epoch 1/10 | Train: 8.364 | Val: 7.549
Epoch 2/10 | Train: 6.795 | Val: 6.326
Epoch 3/10 | Train: 6.162 | Val: 6.186
Epoch 4/10 | 

In [29]:
# ================================
# STEP X: Qualitative Examples
# ================================

def show_qualitative_examples(model, test_loader, urdu_tokenizer, roman_tokenizer, n_batches=2):
    """Show qualitative translations from the model vs ground truth."""
    model.eval()
    examples_shown = 0

    with torch.no_grad():
        for src_batch, trg_batch in test_loader:
            src_batch, trg_batch = src_batch.to(DEVICE), trg_batch.to(DEVICE)

            # Forward pass without teacher forcing
            output = model(src_batch, trg_batch, teacher_forcing_ratio=0)
            pred_ids = output.argmax(-1)

            for j in range(src_batch.size(0)):
                src_text = decode_tokens(src_batch[j], urdu_tokenizer)
                trg_text = decode_tokens(trg_batch[j], roman_tokenizer)
                pred_text = decode_tokens(pred_ids[j], roman_tokenizer)

                print("====================================")
                print(f"📝 Urdu (Source): {src_text}")
                print(f"🎯 Roman (Target): {trg_text}")
                print(f"🤖 Roman (Pred):   {pred_text}")

                examples_shown += 1
                if examples_shown >= n_batches * src_batch.size(0):
                    return  # stop after showing n_batches


# Example usage: show 2 batches of translations from the last loaded model
print("\n📌 Showing qualitative examples from the last model ...\n")
show_qualitative_examples(model, test_loader, urdu_tokenizer, roman_tokenizer, n_batches=2)



📌 Showing qualitative examples from the last model ...

📝 Urdu (Source): عشق میں لا جواب ہیں ہم لوگ 
ماہتاب افتاب ہیں ہم لوگ 
گرچہ اہل شراب ہیں ہم لوگ 
یہ نہ سمجہو خراب ہیں ہم لوگ 
شام سے ا گیے جو پینے پر 
صبح تک افتاب ہیں ہم لوگ 
ہم کو دعوایے عشق بازی ہے 
مستحق عذاب ہیں ہم لوگ 
ناز کرتی ہے خانہ ویرانی 
ایسے خانہ خراب ہیں ہم لوگ 
ہم نہیں جانتے خزاں کیا ہے 
کشتگان شباب ہیں ہم لوگ
🎯 Roman (Target): ishq meñ lā-javāb haiñ ham log 
māhtāb āftāb haiñ ham log 
garche ahl-e-sharāb haiñ ham log 
ye na samjho ḳharāb haiñ ham log 
shaam se aa ga.e jo piine par 
sub.h tak āftāb haiñ ham log 
ham ko dāvā-e-ishq-bāzī hai 
mustahiqq-e-azāb haiñ ham log 
naaz kartī hai ḳhāna-vīrānī 
aise ḳhāna
🤖 Roman (Pred):    jiinedekhodekhodekhodekhodekho taj taj taj taj tajdekhodekhodekhodekhotishtishtishtishtishtishtishtishdekhodekhodekhodekhodekho taj taj taj taj tajdekhodekhodekhodekhotishtishtishtishtishtishtishtishdekhodekhodekhodekhodekho taj taj taj taj tajdekhodekhodekhodekhotishtishtishtishtishtishtish